<a href="https://colab.research.google.com/github/eb-art/DW_road_sign/blob/master/Matrix3_Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_3/DW_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_3/DW_road_sign


In [25]:
!pip install hyperopt

In [26]:
import pandas as pd
import numpy as np

import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_train.shape, y_train.shape

X_test, y_test = test['features'], test['labels']
X_test.shape, y_test.shape

df = pd.read_csv('data/dw_signnames.csv')
labels_dict = df.to_dict()['b']

if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model_param(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', 
              optimizer = 'Adam',
              metrics=['accuracy'])
  
  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  #logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
  #tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
   
  model.fit(
      X_train, y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1) #szuka wiersz po wierszu

  y_test_norm = np.argmax(y_test, axis = 1)

  return scoring(y_test_norm, y_pred) 

In [0]:

def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                    
])


In [0]:
#def train_and_predict(model):
  #model_trained = train_model_param(model, X_train, y_train) 

  #return predict(model_trained, X_test, y_test) 

#train_and_predict(get_cnn_v3(input_shape, num_classes))

In [32]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model_param(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.7049 - accuracy: 0.2740 - val_loss: 1.3049 - val_accuracy: 0.5850
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.0552 - accuracy: 0.6669 - val_loss: 0.3007 - val_accuracy: 0.9050
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.4659 - accuracy: 0.8545 - val_loss: 0.1529 - val_accuracy: 0.9566
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2695 - accuracy: 0.9183 - val_loss: 0.0675 - val_accuracy: 0.9802
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1899 - accuracy: 0.9427 - val_loss: 0.0338 - val_accuracy: 0.9905


0.9752834467120182

In [33]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.0970 - accuracy: 0.9753


0.9752834439277649

In [0]:

def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_cnn_dense_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_cnn_dense_two']),

  Dense(num_classes, activation='softmax')                    
])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', 
              optimizer = 'Adam', metrics=['accuracy'])
  model.fit(
      X_train, y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 7,
      verbose = 0
  )
  accuracy = model.evaluate(X_test, y_test,verbose = 0)[1]
  #print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

space = {
    'batch_size': hp.quniform('batch_size', 10, 200, 10),
    'dropout_cnn_block_one': hp.quniform('dropout_cnn_block_one', 0.3, 0.5, 0.1),
    'dropout_cnn_block_two': hp.quniform('dropout_cnn_block_two', 0.3, 0.5, 0.1),
    'dropout_cnn_block_three': hp.quniform('dropout_cnn_block_three', 0.3, 0.5, 0.1),
    'dropout_cnn_dense_one': hp.quniform('dropout_cnn_dense_one', 0.3, 0.7, 0.1),
    'dropout_cnn_dense_two': hp.quniform('dropout_cnn_dense_two', 0.3, 0.7, 0.1),

  }

  

In [0]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

 73%|███████▎  | 22/30 [08:29<03:23, 25.46s/it, best loss: -0.980272114276886]